# Testing

In [2]:
import duckdb
import plotly.io as pio
import os

import fastf1 as ff1
import pandas as pd
from scipy.spatial import cKDTree

pio.renderers.default = "notebook" 

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)


# Driver minisectors/sectors

In [ ]:
file_path = os.path.abspath("data/driver_timing/2025_1_5_Australian_Grand_Prix_R.parquet")

In [ ]:
drivers_telemetry = duckdb.sql(f"SELECT * FROM '{file_path}'").df()

In [ ]:
#print(drivers_telemetry.Sector.unique())
#print(drivers_telemetry.CornerArea.unique())
#print(drivers_telemetry.DistanceFromPreviousCorner.unique())
#print(drivers_telemetry.DistanceFromPreviousCorner.unique())
print(drivers_telemetry.NextCorner.unique())
#print(drivers_telemetry.PreviousCorner.unique())

In [ ]:
drivers_telemetry.loc[(drivers_telemetry.DriverNumber=='1')].head(50)

In [ ]:
session = ff1.get_session(2025,'Australian_Grand_Prix','FP1')
session.load()

In [ ]:
circuit_info = session.get_circuit_info()
print(len(circuit_info.corners.dropna(subset=["X","Y"])))

In [21]:
file_path = os.path.abspath("data/driver/*_driver_profiles.csv")
drivers = duckdb.sql(f"SELECT * FROM '{file_path}'").df()
drivers

,grand_prix,location,driver,max_throttle_ratio,compound,tyre_age,is_fresh_tyre,avg_rainfall,avg_track_temp,avg_air_temp,braking_events,session_uid,degradation_slope,brake_max_g,brake_avg_g,drs_activations,year,session,event,session_date
0,Australian Grand Prix,Melbourne,VER,0.770552,MEDIUM,2.0,True,0.0,39.300000,23.8,7,2025_Melbourne_FP1,1.649583,4.813682,3.173755,5,2025,FP1,Australian Grand Prix,2025-03-14 01:30:00
1,Australian Grand Prix,Melbourne,NOR,0.735251,SOFT,6.0,True,0.0,39.300000,23.8,8,2025_Melbourne_FP1,-6.608679,5.545173,4.167291,5,2025,FP1,Australian Grand Prix,2025-03-14 01:30:00
2,Australian Grand Prix,Melbourne,BOR,0.747765,SOFT,2.0,True,0.0,39.300000,23.8,8,2025_Melbourne_FP1,-3.604000,5.212946,3.855890,5,2025,FP1,Australian Grand Prix,2025-03-14 01:30:00
3,Australian Grand Prix,Melbourne,HAD,0.750653,SOFT,2.0,True,0.0,39.300000,23.8,10,2025_Melbourne_FP1,10.365600,9.321343,4.418181,5,2025,FP1,Australian Grand Prix,2025-03-14 01:30:00
4,Australian Grand Prix,Melbourne,DOO,0.767907,SOFT,5.0,True,0.0,39.300000,23.8,8,2025_Melbourne_FP1,-8.515821,8.904333,4.349686,5,2025,FP1,Australian Grand Prix,2025-03-14 01:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,British Grand Prix,Silverstone,SAI,0.626580,MEDIUM,11.0,True,0.0,24.260606,18.0,4,2025_Silverstone_R,-2.311845,7.078944,3.712361,0,2025,R,British Grand Prix,2025-07-06 14:00:00
1169,British Grand Prix,Silverstone,HAD,0.528996,INTERMEDIATE,9.0,True,0.0,24.312957,18.0,2,2025_Silverstone_R,4.736600,5.309208,4.073941,0,2025,R,British Grand Prix,2025-07-06 14:00:00
1170,British Grand Prix,Silverstone,RUS,0.678413,HARD,13.0,True,0.0,24.262687,18.0,5,2025_Silverstone_R,-2.116248,4.028186,2.663007,0,2025,R,British Grand Prix,2025-07-06 14:00:00
1171,British Grand Prix,Silverstone,PIA,0.673167,MEDIUM,8.0,True,0.0,24.258498,18.0,3,2025_Silverstone_R,-2.835950,3.282056,2.922746,0,2025,R,British Grand Prix,2025-07-06 14:00:00


In [20]:
file_path = os.path.abspath("data/circuit/*_circuit_profiles.csv")
circuit = duckdb.sql(f"SELECT * FROM '{file_path}'").df()
circuit

,year,event,location,session,real_altitude,lap_length,telemetry_source,avg_speed,top_speed,braking_events,low_pct,med_pct,high_pct,slow_corners,medium_corners,fast_corners,chicanes,air_temp_avg,track_temp_avg,rain_detected
0,2025,Australian Grand Prix,Melbourne,FP1,2.0,5222.361111,fastf1,241.778157,325.0,8,0.068259,0.180887,0.750853,0,4,10,7,24.403704,42.076543,False
1,2025,Australian Grand Prix,Melbourne,FP2,2.0,5222.180278,fastf1,246.030822,323.0,9,0.047945,0.184932,0.767123,0,4,10,7,23.760494,41.132099,False
2,2025,Australian Grand Prix,Melbourne,FP3,2.0,5207.668889,fastf1,249.886926,325.0,12,0.049470,0.162544,0.787986,0,4,10,5,29.492308,40.801282,False
3,2025,Australian Grand Prix,Melbourne,Q,2.0,5208.930833,fastf1,250.118705,329.0,11,0.046763,0.172662,0.780576,0,4,10,5,30.336000,40.501333,False
4,2025,Australian Grand Prix,Melbourne,R,2.0,5219.984167,fastf1,229.514658,302.0,5,0.074919,0.208469,0.716612,1,4,9,7,15.707865,18.942135,True
5,2025,Chinese Grand Prix,Shanghai,FP1,4.0,5372.565000,fastf1,210.643713,332.0,10,0.176647,0.236527,0.586826,3,5,8,7,22.764634,36.152439,False
6,2025,Chinese Grand Prix,Shanghai,SQ,4.0,5404.225556,fastf1,213.489489,337.0,11,0.171171,0.252252,0.576577,3,5,8,7,24.776667,36.803333,False
7,2025,Chinese Grand Prix,Shanghai,S,4.0,5360.830278,fastf1,202.533898,314.0,4,0.189266,0.279661,0.531073,4,5,7,8,22.896429,35.660714,False
8,2025,Chinese Grand Prix,Shanghai,Q,4.0,5377.302778,fastf1,214.777108,330.0,8,0.159639,0.237952,0.602410,2,6,8,7,26.132468,38.670130,False
9,2025,Chinese Grand Prix,Shanghai,R,4.0,5392.193889,fastf1,203.430939,316.0,5,0.209945,0.267956,0.522099,4,5,7,7,27.215584,35.816883,False


In [3]:
from helpers.driver_utils import get_driver_max_throttle_ratio
from fastf1 import get_session


##### LOADED get_driver_max_throttle_ratio from /Users/tomasz.solis/repos/private/f1project/formula1/helpers/driver_utils.py
DEBUG: [DRIVER-FEATURE ERROR] VER @ Australian Grand Prix FP1 → "['DriverAhead', 'DistanceToDriverAhead'] not in index"


(None,
               grand_prix   location driver         session_uid
 0  Australian Grand Prix  Melbourne    VER  2025_Melbourne_FP1)

In [18]:
def get_driver_max_throttle_ratio(session, 
                                  driver, 
                                  max_throttle_threshold: int = 98, 
                                  season: int | None = None,
                                  session_name: str | None = None,):    
    
    """
    Calculate the full throttle ratio for a driver's fastest lap.

    Parameters:
    - session: FastF1 loaded session
    - driver: str, driver code (e.g., 'VER')
    - max_throttle_threshold: float, minimum throttle to be considered full throttle
    - season: optional, int
    - session_name: optional, str (e.g., 'FP1')

    Returns:
    - result: pd.DataFrame with telemetry and weather metrics
    - missing_info: pd.DataFrame fallback if telemetry unavailable
    """
    #print("##### LOADED get_driver_max_throttle_ratio from", __file__)
    gp_name = session.event['EventName']
    location = session.event['Location']

    #try:
    fastest_driver = session.laps.pick_drivers([driver]).pick_fastest()
    telemetry = fastest_driver.get_car_data().add_distance()

    total_dist = telemetry["Distance"].max()
    if total_dist > 0:
        telemetry["RelativeDistance"] = telemetry["Distance"] / total_dist
    else:
        telemetry["RelativeDistance"] = 0

    # *** DEBUG INSPECTION ***
    print(f"[DEBUG] {driver} @ {gp_name} {session_name} — telemetry columns:", telemetry.columns.tolist())
    print(f"[DEBUG] {driver} @ {gp_name} {session_name} — rows in telemetry:", len(telemetry))

    telemetry = pd.merge_asof(
        telemetry,
        session.weather_data[['Time', 'Rainfall', 'TrackTemp', 'AirTemp']],
        on='Time'
    )

    telemetry['delta_speed'] = telemetry['Speed'].diff()
    heavy_brakes = telemetry['delta_speed'] < -30
    braking_events = heavy_brakes.sum()

    telemetry['nextThrottle'] = telemetry.Throttle.shift(-1)
    telemetry['previousThrottle'] = telemetry.Throttle.shift(1)

    throttle_points = telemetry.loc[
        (telemetry.Throttle >= max_throttle_threshold) &
        (
            (telemetry.nextThrottle < max_throttle_threshold) |
            (telemetry.previousThrottle < max_throttle_threshold) |
            (telemetry.index.isin([telemetry.index[0], telemetry.index[-1]]))
            )
    ].copy()

    print(f"[DEBUG] {driver} @ {gp_name} {session_name} — throttle_points rows:", len(throttle_points))
    if "RelativeDistance" in throttle_points:
        print(f"[DEBUG] {driver} @ {gp_name} {session_name} — sample RelativeDistance diffs:",
                throttle_points.RelativeDistance.diff().head())
            
    throttle_points['FTRelative'] = throttle_points.RelativeDistance.diff().fillna(0)

    max_throttle_ratio = throttle_points.loc[
        (throttle_points.nextThrottle < max_throttle_threshold) |
        (throttle_points.nextThrottle.isna())
    ]['FTRelative'].sum()

    result = pd.DataFrame([{
        'grand_prix': gp_name,
        'location': location,
        'driver': driver,
        'max_throttle_ratio': max_throttle_ratio,
        'compound': fastest_driver['Compound'],
        'tyre_age': fastest_driver['TyreLife'],
        'is_fresh_tyre': fastest_driver['FreshTyre'],
        'avg_rainfall': telemetry['Rainfall'].mean(),
        'avg_track_temp': telemetry['TrackTemp'].mean(),
        'avg_air_temp': telemetry['AirTemp'].mean(),
        'braking_events': braking_events,
        'session_uid': f"{season}_{location}_{session_name}" if season and session_name else None
        }])

   
    return result


In [19]:
sess = get_session(2025, "Australian Grand Prix", "FP1")
sess.load(telemetry=True, laps=True, weather=True)
get_driver_max_throttle_ratio(sess, "VER", season=2025, session_name="FP1")

[DEBUG] VER @ Australian Grand Prix FP1 — telemetry columns: ['Date', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source', 'Time', 'SessionTime', 'Distance', 'RelativeDistance']
[DEBUG] VER @ Australian Grand Prix FP1 — rows in telemetry: 285
[DEBUG] VER @ Australian Grand Prix FP1 — throttle_points rows: 23
[DEBUG] VER @ Australian Grand Prix FP1 — sample RelativeDistance diffs: 0          NaN
9     0.036788
18    0.030916
47    0.105756
65    0.039009
Name: RelativeDistance, dtype: float64


,grand_prix,location,driver,max_throttle_ratio,compound,tyre_age,is_fresh_tyre,avg_rainfall,avg_track_temp,avg_air_temp,braking_events,session_uid
0,Australian Grand Prix,Melbourne,VER,0.770552,SOFT,2.0,True,0.0,39.3,23.8,7,2025_Melbourne_FP1
